<a href="https://colab.research.google.com/github/iplusplus42/haiku-gen/blob/main/Haiku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# NOTE: don't run this cell on Colab, it might break something. It loads my code formatter locally.
%load_ext blackcellmagic

In [3]:
import random
import os
from urllib import request
import zipfile
from typing import List

DEBUG = False
LOCAL = os.path.exists(os.path.join(os.getcwd(), ".gitignore"))


class Utils:
    @staticmethod
    def is_palindrome(string):
        return string == string[::-1]

    @staticmethod
    def get_files():
        with request.urlopen("https://insertplus.keybase.pub/words.zip") as f:
            fd = open("words.zip", "wb")
            fd.write(f.read())
            fd.close()

            fd = open("words.zip", "rb")
            words_zip = zipfile.ZipFile(fd)
            words_zip.extractall()
            words_zip.close()
            fd.close()

    @staticmethod
    def get_adjectives(syllable=1):
        adjectives = set()
        path = os.path.join(os.getcwd(), "words/adjectives")

        if syllable in range(1, 5):
            fd = open(os.path.join(path, f"{syllable}syllableadjectives.txt"))
            words = [i for i in fd.read().split("\n") if i != ""]
            adjectives.update({i for i in words})
        else:
            for i in os.listdir(path):
                fd = open(os.path.join(path, i), "r")
                words = [i for i in fd.read().split("\n") if i != ""]
                adjectives.update({i for i in words})
                fd.close()
        return adjectives

    @staticmethod
    def get_adj(syllable=1):
        return random.choice([i for i in Utils.get_adjectives(syllable)])

    get_adjective = get_adj

    @staticmethod
    def get_adverbs(syllable=1):
        adverbs = set()
        path = os.path.join(os.getcwd(), "words/adverbs")

        if syllable in range(1, 5):
            fd = open(os.path.join(path, f"{syllable}syllableadverbs.txt"))
            words = [i for i in fd.read().split("\n") if i != ""]
            adverbs.update({i for i in words})
        else:
            for i in os.listdir(path):
                fd = open(os.path.join(path, i), "r")
                words = [i for i in fd.read().split("\n") if i != ""]
                adverbs.update({i for i in words})
                fd.close()
        return adverbs

    @staticmethod
    def get_adverb(syllable=1):
        return random.choice([i for i in Utils.get_adverbs(syllable)])

    @staticmethod
    def get_nouns(syllable=1):
        nouns = set()
        path = os.path.join(os.getcwd(), "words/nouns")

        if syllable in range(1, 5):
            fd = open(os.path.join(path, f"{syllable}syllablenouns.txt"))
            words = [i for i in fd.read().split("\n") if i != ""]
            nouns.update({i for i in words})
        else:
            for i in os.listdir(path):
                fd = open(os.path.join(path, i), "r")
                words = [i for i in fd.read().split("\n") if i != ""]
                nouns.update({i for i in words})
                fd.close()
        return nouns

    @staticmethod
    def get_noun(syllable=1):
        return random.choice([i for i in Utils.get_nouns(syllable)])

    @staticmethod
    def get_verbs(syllable=1):
        verbs = set()
        path = os.path.join(os.getcwd(), "words/verbs")

        if syllable in range(1, 5):
            fd = open(os.path.join(path, f"{syllable}syllableverbs.txt"))
            words = [i for i in fd.read().split("\n") if i != ""]
            verbs.update({i for i in words})
        else:
            for i in os.listdir(path):
                fd = open(os.path.join(path, i), "r")
                words = [i for i in fd.read().split("\n") if i != ""]
                verbs.update({i for i in words})
                fd.close()
        return verbs

    @staticmethod
    def get_verb(syllable=1):
        return random.choice([i for i in Utils.get_verbs(syllable)])

    @staticmethod
    def get_words(syllable=1):
        return random.choice([Utils.get_verbs, Utils.get_nouns, Utils.get_adverbs, Utils.get_adjectives])(syllable)

    @staticmethod
    def get_word(syllable=1):
        return random.choice(
            [
                i
                for i in random.choice([Utils.get_verbs, Utils.get_nouns, Utils.get_adverbs, Utils.get_adjectives])(
                    syllable
                )
            ]
        )


class Rules:
    # TODO: We could probably simplify this in the future, by perhaps writing a function that takes total_syllables:int
    # and elements:List[str], where elements is used to place adjectives, nouns, and verbs into their respective positions.
    @staticmethod
    def do_adj_noun_verb(total_syllables: int):
        seed: List[str] = ["adj", "noun", "verb"]
        random.shuffle(
            seed
        )  # the seed determines which of the three gets priority when multiple categories get the same syllable count
        rand_max: int = 3 if total_syllables == 5 else 4
        syllables = {
            "adj": random.randint(1, rand_max),
            "noun": random.randint(1, rand_max),
            "verb": random.randint(1, rand_max),
        }

        if (syllables["adj"] + syllables["noun"] + syllables["verb"]) > total_syllables:
            debug("gt")
            while (int(syllables[seed[0]]) + int(syllables[seed[1]]) + int(syllables[seed[2]])) > total_syllables:
                debug(f"{syllables[seed[0]]} {syllables[seed[1]]} {syllables[seed[2]]}")
                if syllables[seed[2]] >= 2:
                    syllables[seed[2]] -= 1
                elif syllables[seed[1]] >= 2:
                    syllables[seed[1]] -= 1
                elif syllables[seed[0]] >= 2:
                    syllables[seed[0]] -= 1
            return f"{Utils.get_adj(syllables['adj'])} {Utils.get_noun(syllables['noun'])} {Utils.get_verb(syllables['verb'])}"
        elif (syllables["adj"] + syllables["noun"] + syllables["verb"]) == total_syllables:
            debug("eq")
            return f"{Utils.get_adj(syllables['adj'])} {Utils.get_noun(syllables['noun'])} {Utils.get_verb(syllables['verb'])}"
        else:
            debug("lt")
            while (syllables[seed[0]] + syllables[seed[1]] + syllables[seed[2]]) < total_syllables:
                debug(f"{syllables[seed[0]]} {syllables[seed[1]]} {syllables[seed[2]]}")
                syllables[seed[0]] += 1
            return f"{Utils.get_adj(syllables['adj'])} {Utils.get_noun(syllables['noun'])} {Utils.get_verb(syllables['verb'])}"

    @staticmethod
    def do_verb_adj_noun(total_syllables: int):
        seed: List[str] = ["adj", "noun", "verb"]
        random.shuffle(
            seed
        )  # the seed determines which of the three gets priority when multiple categories get the same syllable count
        rand_max: int = 3 if total_syllables == 5 else 4
        syllables = {
            "adj": random.randint(1, rand_max),
            "noun": random.randint(1, rand_max),
            "verb": random.randint(1, rand_max),
        }

        if (syllables["adj"] + syllables["noun"] + syllables["verb"]) > total_syllables:
            debug("gt")
            while (int(syllables[seed[0]]) + int(syllables[seed[1]]) + int(syllables[seed[2]])) > total_syllables:
                debug(f"{syllables[seed[0]]} {syllables[seed[1]]} {syllables[seed[2]]}")
                if syllables[seed[2]] >= 2:
                    syllables[seed[2]] -= 1
                elif syllables[seed[1]] >= 2:
                    syllables[seed[1]] -= 1
                elif syllables[seed[0]] >= 2:
                    syllables[seed[0]] -= 1
            return f"{Utils.get_adj(syllables['verb'])} {Utils.get_noun(syllables['adj'])} {Utils.get_verb(syllables['noun'])}"
        elif (syllables["adj"] + syllables["noun"] + syllables["verb"]) == total_syllables:
            debug("eq")
            return f"{Utils.get_adj(syllables['verb'])} {Utils.get_noun(syllables['adj'])} {Utils.get_verb(syllables['noun'])}"
        else:
            debug("lt")
            while (syllables[seed[0]] + syllables[seed[1]] + syllables[seed[2]]) < total_syllables:
                debug(f"{syllables[seed[0]]} {syllables[seed[1]]} {syllables[seed[2]]}")
                syllables[seed[0]] += 1
            return f"{Utils.get_adj(syllables['verb'])} {Utils.get_noun(syllables['adj'])} {Utils.get_verb(syllables['noun'])}"

    @staticmethod
    def do_adj_noun(total_syllables: int):
        seed: List[str] = ["adj", "noun"]
        random.shuffle(
            seed
        )  # the seed determines which of the three gets priority when multiple categories get the same syllable count
        rand_max: int =  4
        syllables = {
            "adj": random.randint(1, rand_max),
            "noun": random.randint(1, rand_max)
        }

        if (syllables["adj"] + syllables["noun"]) > total_syllables:
            debug("gt")
            while (int(syllables[seed[0]]) + int(syllables[seed[1]])) > total_syllables:
                debug(f"{syllables[seed[0]]} {syllables[seed[1]]}")
                if syllables[seed[1]] >= 2:
                    syllables[seed[1]] -= 1
                elif syllables[seed[0]] >= 2:
                    syllables[seed[0]] -= 1
            return f"{Utils.get_adj(syllables['adj'])} {Utils.get_noun(syllables['noun'])}"
        elif (syllables["adj"] + syllables["noun"]) == total_syllables:
            debug("eq")
            return f"{Utils.get_adj(syllables['adj'])} {Utils.get_noun(syllables['noun'])}"
        else:
            debug("lt")
            while (syllables[seed[0]] + syllables[seed[1]]) < total_syllables:
                debug(f"{syllables[seed[0]]} {syllables[seed[1]]}")
                syllables[seed[0]] += 1
            return f"{Utils.get_adj(syllables['adj'])} {Utils.get_noun(syllables['noun'])}"

    @staticmethod
    def do_noun_verb(total_syllables: int):
        seed: List[str] = ["noun", "verb"]
        random.shuffle(
            seed
        )  # the seed determines which of the three gets priority when multiple categories get the same syllable count
        rand_max: int = 4
        syllables = {
            "noun": random.randint(1, rand_max),
            "verb": random.randint(1, rand_max),
        }

        if (syllables["noun"] + syllables["verb"]) > total_syllables:
            debug("gt")
            while (int(syllables[seed[0]]) + int(syllables[seed[1]])) > total_syllables:
                debug(f"{syllables[seed[0]]} {syllables[seed[1]]}")
                if syllables[seed[1]] >= 2:
                    syllables[seed[1]] -= 1
                elif syllables[seed[0]] >= 2:
                    syllables[seed[0]] -= 1
            return f"{Utils.get_noun(syllables['noun'])} {Utils.get_verb(syllables['verb'])}"
        elif (syllables["noun"] + syllables["verb"]) == total_syllables:
            debug("eq")
            return f"{Utils.get_noun(syllables['noun'])} {Utils.get_verb(syllables['verb'])}"
        else:
            debug("lt")
            while (syllables[seed[0]] + syllables[seed[1]]) < total_syllables:
                debug(f"{syllables[seed[0]]} {syllables[seed[1]]}")
                syllables[seed[0]] += 1
            return f"{Utils.get_noun(syllables['noun'])} {Utils.get_verb(syllables['verb'])}"

    @staticmethod
    def random_rule(total_syllables: int):
        return random.choice([getattr(Rules, i) for i in Rules().__dir__() if i.startswith("do_")])(total_syllables)


def debug(content):
    if DEBUG:
        print(content)


def absurd_haiku():
    line_1_syllables = 5
    line_2_syllables = 7
    line_3_syllables = 5
    temp = {"line1": "", "line2": "", "line3": ""}

    debug("Line 1")
    while line_1_syllables > 0:
        syllable = random.randint(1, 4 if line_1_syllables > 4 else line_1_syllables)
        debug(f"Remaining syllables: {line_1_syllables}")
        debug(f"Syllable: {syllable}")
        if syllable > line_1_syllables:
            continue
        word = Utils.get_word(syllable)
        debug(f"Word: {word}")
        temp["line1"] += f"{word} "
        line_1_syllables -= syllable

    debug("Line 2")
    while line_2_syllables > 0:
        syllable = random.randint(1, 4 if line_2_syllables > 4 else line_2_syllables)
        debug(f"Remaining syllables: {line_2_syllables}")
        debug(f"Syllable: {syllable}")
        if syllable > line_2_syllables:
            continue
        word = Utils.get_word(syllable)
        debug(f"Word: {word}")
        temp["line2"] += f"{word} "
        line_2_syllables -= syllable

    debug("Line 3")
    while line_3_syllables > 0:
        syllable = random.randint(1, 4 if line_3_syllables > 4 else line_3_syllables)
        debug(f"Remaining syllables: {line_3_syllables}")
        debug(f"Syllable: {syllable}")
        if syllable > line_3_syllables:
            continue
        word = Utils.get_word(syllable)
        debug(f"Word: {word}")
        temp["line3"] += f"{word} "
        line_3_syllables -= syllable

    return f"{temp['line1']}\n{temp['line2']}\n{temp['line3']}"


if __name__ == "__main__":
    if not LOCAL:
        Utils.get_files()
    print("HaikuGenerator v0.2.0")
    print("Absurd haiku")
    print(absurd_haiku())
    print("\nAdjective noun verb rule")
    print(Rules.do_adj_noun_verb(5))
    print(Rules.do_adj_noun_verb(7))
    print(Rules.do_adj_noun_verb(5))
    print("\nAdjective noun rule")
    print(Rules.do_adj_noun(5))
    print(Rules.do_adj_noun(7))
    print(Rules.do_adj_noun(5))
    print("\nNoun verb rule")
    print(Rules.do_noun_verb(5))
    print(Rules.do_noun_verb(7))
    print(Rules.do_noun_verb(5))
    print("\nVerb adjective noun")
    print(Rules.do_verb_adj_noun(5))
    print(Rules.do_verb_adj_noun(7))
    print(Rules.do_verb_adj_noun(5))
    print("\nRandom rules")
    print(Rules.random_rule(5))
    print(Rules.random_rule(7))
    print(Rules.random_rule(5))

HaikuGenerator v0.2.0
Absurd haiku
reselect chance wedged 
gyrally deign nomism damn 
plywoods milliares 

Adjective noun verb rule
ult mavourneen scythe
arguable drip screech
hooked murrains misdoubts

Adjective noun rule
crutched barbiturate
disquiet chechakoes
frore indulgencies

Noun verb rule
saltiness wintles
penetrances rallying
clostridia sates

Verb adjective noun
mighty hasp sojourns
foziest ligands gestates
dovish guaco starts

Random rules
keckling upstaging
bevelers sterilizes
trig disrelishes
